# Data Engineering Capstone Project

#### Project Summary
This project will create a data model that an analytics team can use to gain insights into United States immigration data. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Conclusion and Notes

In [1]:
#install if do not have pycountry
!pip install pycountry

    100% |████████████████████████████████| 10.0MB 2.9MB/s eta 0:00:01   22% |███████▍                        | 2.3MB 29.4MB/s eta 0:00:01    54% |█████████████████▋              | 5.5MB 34.7MB/s eta 0:00:01    86% |███████████████████████████▊    | 8.7MB 36.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for pycountry ... done
  Stored in directory: /root/.cache/pip/wheels/a2/98/bf/f0fa1c6bf8cf2cbdb750d583f84be51c2cd8272460b8b36bd3
Successfully built pycountry


In [3]:
# Imports and installs
import pandas as pd
import numpy as np
import os
import glob
import pycountry
from datetime import datetime, timedelta
import pyspark.sql.functions as f
from pyspark.sql import types as T
from pyspark.sql.functions import udf, col,  trim, initcap, year, month, upper, round, substring, length, when
from pyspark.sql import SparkSession

from immigration_countries import i94country_name
from countries import get_country_code
from i94_states import get_state_name, get_state_code

# Create spark session
spark = SparkSession\
.builder\
.config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11") \
.getOrCreate()

# SQL context
from pyspark.sql import SQLContext
sc = spark.sparkContext
sqlContext = SQLContext(sc)

### Project Scope and Data
The objective of this project is to create a data pipeline that will combine datasets from multiple sources into a data model with a star schema. The data model shoud allow users to analyze immigration trends and other descriptive statistics. Analysts should be able to use infomration outside of the immigration dataset, such as data about temperature, US city demographics, and airport data to perform their analyses.

To create the star schema model, I will implment a data pipeline that processes data from multiple sources. The data sources I am using can be found at the following links:
- [__I94 immigration data:__](https://travel.trade.gov/research/reports/i94/historical/2016.html) US immigration data from the US National Tourism and Trade Office 

- [__World temperature data:__](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data) Contains CSV files for overall global temperatures and temperatures by city, major city, state, country.

- [__US city demographics data:__](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) Demographics data includes values such as population, race, age, and household size.

- [__Airport codes data:__](https://datahub.io/core/airport-codes#data) Contains data related to airports. Airports have three character codes that uniquely identify them. For example, LaGuardia airport's code is LGA.   

Using the final data model should allow analysts to answer questions such as:
- How many people immigrated from a certain country?
- Which US city had the most immigrants in a given time period?
- How many people immigrated to each state?
- How many immigrants came under each type of visa (student, business, pleasure)?

### Step 2: Explore, Assess, and Clean the Data

#### Exploratory Data Anlysis (EDA)

I performed exploratory data analysis to determine what is in each file (data types, what each column represents, number or records) and identify any data transformations that need to be performed.

EDA was performed on all 4 data sources in separate Python notebook using Pandas. Key takeaways for each dataset:

_Immigration data:_
 - 29 columns
 - Columns that are integers are being read in as floats. Change these datatypes into integers when creating data model
 - Dates are not date types. They are also based on SAS dates, which will require extra processing


_Demographics data:_
- 12 columns
- Some columns are floats when they should be integers
- Each city has multiple rows of data. For example Birmingham, Alabama has five rows. In each <br>of these rows there are repeat values for most of the columns. Data should be transposed/pivoted so there is only one row per city.
 - For example, there are 5 race categoires (White, Black, Asian, Hispanic or Latio, and ). The dataset will have separate rows for each <br>of these races for every city. If we transpose/pivot the data, then each row will correspond with 1 city with values in each race column.

_Airport data:_
- 12 columns
- Can derive continent from region column. 
- Can also derive country from the region column. Good since region has no null values, but country has some nulls
- Many rows had nulls in the continent and iso_country columns. This is because the values are "NA". We need to process <br>these with escape characters to prevent improper reading of this field. 

_Temperature data:_
- 5 columns
- AverageTemperature represents the average temperature for a given month. This is recorded in a date column with a date value for the first of the month
- Add a country code and state code that uses a 2 digit ISO code for uniform naming. This will allow us to link/join datasets
 - Need to do a little customization/manual linking for some countries due to differing naming conventions between the dataset and standard lookup directory


#### Cleaning Steps

Before creating an ETL scirpt, identify what steps are needed to clean and transform the data into what we want to use in our data model:
1. Immigration
- Convert the following columns from float/double to integer: i94yr, i94mo, i94cit, i94res, i94mode, i94bir, i94visa
- Convert date in arrdate and depdate. Currently in SAS date format and need to convert to a date type object that reflects the correct date
2. Demographic data
- Convert the following columns from float/double to integer: Male Population, Female Population, Total Population, Total Population, Number of Veterans, Foreign-born
- Transpose demographics data
 - Each row represents a single city
 - Add columns for each "Race" value
 - Add percent columns for all of the count columns based on the total population of the city. Since cities have differing sizes, it would be good to view these as percentages
3. Aiport data
-  Add a columns that has a 2 digit ISO country code for uniform naming. This will allow us to link/join datasets
- Need to do a little customization/manual linking for some countries due to differing naming conventions between the dataset and standard lookup directory (see EDA 
4. Temperature data
- Add a columns that has a 2 digit ISO country code for uniform naming. This will allow us to link/join datasets
- Need to do a little customization/manual linking for some countries due to differing naming conventions between the dataset and standard lookup directory (see EDA steps)

#### 1) Immigration Data Cleaning
- Convert from float/double to integer: i94yr, i94mo, i94cit, i94res, i94mode, i94bir, i94visa
- Convert date in arrdate and depdate. Currently in SAS date format and need to convert to regular

In [4]:
#import data into spark dataframes
immigration_spark = spark.read.load('./sas_data/*')

In [5]:
immigration_spark.show(2)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

In [6]:
immigration_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [7]:
# Define a function that can handle converting SAS dates
def convert_datetime(x):
    try:
        start = datetime(1960, 1, 1)
        return start + timedelta(days=int(x))
    except:
        return None

datetime_from_sas = udf(lambda x: convert_datetime(x), T.DateType())

In [8]:
# Convert data types and process SAS dates
# This takes in the i94res column, and uses the 3 digit number to get a name for the country.
# It uses the provided data dictionary from the immigration dataset

immigration_clean = immigration_spark \
    .withColumn('cicid', immigration_spark['cicid'].cast('integer')) \
    .withColumn('i94yr', immigration_spark['i94yr'].cast('integer')) \
    .withColumn('i94mon', immigration_spark['i94mon'].cast('integer')) \
    .withColumn('i94cit', immigration_spark['i94cit'].cast('integer')) \
    .withColumn('i94res', immigration_spark['i94res'].cast('integer')) \
    .withColumn('i94mode', immigration_spark['i94mode'].cast('integer')) \
    .withColumn('i94bir', immigration_spark['i94bir'].cast('integer')) \
    .withColumn('i94visa', immigration_spark['i94visa'].cast('integer')) \
    .withColumn('arrival_date', datetime_from_sas(col('arrdate'))) \
    .withColumn('departure_date', datetime_from_sas('depdate')) \
    .withColumn('i94res_name', trim(i94country_name('i94res'))) \
    .withColumn('i94res_isocode', get_country_code(upper(col('i94res_name'))))\
    .withColumn('i94addr_name', get_state_name(col('i94addr')))\
    .withColumn('trip_duration', (col('depdate') - col('arrdate')).cast('integer'))
    
    

In [9]:
immigration_clean.dtypes

[('cicid', 'int'),
 ('i94yr', 'int'),
 ('i94mon', 'int'),
 ('i94cit', 'int'),
 ('i94res', 'int'),
 ('i94port', 'string'),
 ('arrdate', 'double'),
 ('i94mode', 'int'),
 ('i94addr', 'string'),
 ('depdate', 'double'),
 ('i94bir', 'int'),
 ('i94visa', 'int'),
 ('count', 'double'),
 ('dtadfile', 'string'),
 ('visapost', 'string'),
 ('occup', 'string'),
 ('entdepa', 'string'),
 ('entdepd', 'string'),
 ('entdepu', 'string'),
 ('matflag', 'string'),
 ('biryear', 'double'),
 ('dtaddto', 'string'),
 ('gender', 'string'),
 ('insnum', 'string'),
 ('airline', 'string'),
 ('admnum', 'double'),
 ('fltno', 'string'),
 ('visatype', 'string'),
 ('arrival_date', 'date'),
 ('departure_date', 'date'),
 ('i94res_name', 'string'),
 ('i94res_isocode', 'string'),
 ('i94addr_name', 'string'),
 ('trip_duration', 'int')]

In [10]:
# Selecting columns
immigration_clean = immigration_clean\
    .select(col('cicid').alias('id'), col('i94mon').alias('month'), col('i94yr').alias('year'),\
            col('arrival_date'), col('departure_date'), col('trip_duration').cast('integer'),\
            col('i94res_isocode').alias('origin_country_code'),\
            col('i94res_name').alias('origin_country'), col('i94mode').alias('transportation_mode'),\
            col('i94addr').alias('us_state'), col('i94addr_name').alias('us_state_name'),\
            col('i94port').alias('port'), col('gender'), col('i94bir').alias('age'), col('airline'),\
            col('i94visa').alias('visa_type'))

In [11]:
immigration_clean.show(2)

+-------+-----+----+------------+--------------+-------------+-------------------+--------------+-------------------+--------+-------------+----+------+---+-------+---------+
|     id|month|year|arrival_date|departure_date|trip_duration|origin_country_code|origin_country|transportation_mode|us_state|us_state_name|port|gender|age|airline|visa_type|
+-------+-----+----+------------+--------------+-------------+-------------------+--------------+-------------------+--------+-------------+----+------+---+-------+---------+
|5748517|    4|2016|  2016-04-30|    2016-05-08|            8|                 AU|     AUSTRALIA|                  1|      CA|   California| LOS|     F| 40|     QF|        1|
|5748518|    4|2016|  2016-04-30|    2016-05-17|           17|                 AU|     AUSTRALIA|                  1|      NV|       Nevada| LOS|     F| 32|     VA|        1|
+-------+-----+----+------------+--------------+-------------+-------------------+--------------+-------------------+--------

In [12]:
immigration_clean.printSchema()

root
 |-- id: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- trip_duration: integer (nullable = true)
 |-- origin_country_code: string (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- transportation_mode: integer (nullable = true)
 |-- us_state: string (nullable = true)
 |-- us_state_name: string (nullable = true)
 |-- port: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- visa_type: integer (nullable = true)



In [13]:
# Create temp view to use for Spark SQL
immigration_clean.createOrReplaceTempView('immigration')

In [14]:
# How many rows are in the data
spark.sql(
"""
SELECT COUNT(*) as TotalRows
FROM immigration
"""
).show()

# How many months are contained in the data?
spark.sql(
"""
SELECT month, COUNT(*) as Count
FROM immigration
GROUP BY month
"""
).show(30)

# How many years are contained in the data?
spark.sql(
"""
SELECT year, COUNT(*) as Count
FROM immigration
GROUP BY year
"""
).show()

+---------+
|TotalRows|
+---------+
|  3096313|
+---------+

+-----+-------+
|month|  Count|
+-----+-------+
|    4|3096313|
+-----+-------+

+----+-------+
|year|  Count|
+----+-------+
|2016|3096313|
+----+-------+



#### 2) Demographics Data Cleaning

Convert the collowing columns from float/double to integer: 
- Male Population, Female Population, Total Population, Total Population, Number of Veterans, Foreign-born

Aggregate demographics data:

- Each row represents a single city's count for the number a certain race. This results in cities having many rows.
- By aggregating the data on race, an individual row can represent one city by having different race columns

Add percent columns for all of the count columns based on the total population of the city. Since cities have differing sizes, it would be good to view these as percentages as well

In [15]:
# Import demographics data into Spark dataframe
demo_spark = spark.read.csv('./us-cities-demographics.csv', header = True, sep = ';', inferSchema = True)

In [16]:
demo_spark.printSchema()
#demo_spark.show(5)

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



In [17]:
demo_pivot = demo_spark.groupBy('City','State','Median Age','Male Population', 'Female Population', 'Total Population', 'Number of Veterans',
    'Foreign-born', 'Average Household Size', 'State Code')\
    .pivot("Race").sum('Count')

In [18]:
demo_pivot.show(2)

+----------+------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+---------------------------------+-----+-------------------------+------------------+------+
|      City|       State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|American Indian and Alaska Native|Asian|Black or African-American|Hispanic or Latino| White|
+----------+------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+---------------------------------+-----+-------------------------+------------------+------+
|   Modesto|  California|      35.2|         104852|           106405|          211257|              9855|       39613|                  2.97|        CA|                             4388|19417|                     9869|             85141|166618|
|Pittsburgh|Penn

In [19]:
demo_pivot.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- American Indian and Alaska Native: long (nullable = true)
 |-- Asian: long (nullable = true)
 |-- Black or African-American: long (nullable = true)
 |-- Hispanic or Latino: long (nullable = true)
 |-- White: long (nullable = true)



In [20]:
demo1 = demo_pivot\
    .groupBy('State','State Code')\
    .agg(f.sum('Male Population').alias('MalePopulation'),\
         f.sum('Female Population').alias('FemalePopulation'),\
         f.sum('Total Population').alias('TotalPopulation'),\
         f.sum('Number of Veterans').alias('NumberVeterans'),\
         f.sum('Foreign-born').alias('Foreign-born'),\
         f.sum('American Indian and Alaska Native').alias('AmericanIndianAlaskaNativeTotal'),\
         f.sum('Asian').alias('AsianTotal'),\
         f.sum('Black or African-American').alias('BlackTotal'),\
         f.sum('Hispanic or Latino').alias('HispanicTotal'),\
         f.sum('White').alias('WhiteTotal'),\
         f.avg('Median Age').alias('MedianAge'),\
         f.avg('Average Household Size').alias('AvgHouseholdSize'))

In [21]:
demo1.show(2)

+-----------+----------+--------------+----------------+---------------+--------------+------------+-------------------------------+----------+----------+-------------+----------+---------+------------------+
|      State|State Code|MalePopulation|FemalePopulation|TotalPopulation|NumberVeterans|Foreign-born|AmericanIndianAlaskaNativeTotal|AsianTotal|BlackTotal|HispanicTotal|WhiteTotal|MedianAge|  AvgHouseholdSize|
+-----------+----------+--------------+----------------+---------------+--------------+------------+-------------------------------+----------+----------+-------------+----------+---------+------------------+
|Mississippi|        MS|        112147|          130536|         242683|         14792|        4861|                            323|      2587|    167366|         7264|     71645|     33.4|2.5949999999999998|
|       Utah|        UT|        530818|          519773|        1050591|         39671|      132819|                          18746|     48801|     21893|       201

In [22]:
demo2 = demo1\
    .select(col('State'), col('State Code').alias('StateCode'), col('MedianAge'), col('MalePopulation'),\
    col('FemalePopulation'),col('TotalPopulation'), col('NumberVeterans'),col('Foreign-born'),\
    col("AvgHouseholdSize"), col('AmericanIndianAlaskaNativeTotal'), col('AsianTotal'), col('BlackTotal'),\
    col('HispanicTotal'), col('WhiteTotal'))\
    .withColumn('MalePopPercent', round(col('MalePopulation') / col('TotalPopulation'), 4))\
    .withColumn('FemalePopPercent', round(col('FemalePopulation') / col('TotalPopulation'), 4))\
    .withColumn('VeteranPopPercent', round(col('NumberVeterans') / col('TotalPopulation'), 4))\
    .withColumn('ForeignPopPercent', round(col('Foreign-born') / col('TotalPopulation'), 4))\
    .withColumn('Black_pct', round(col('BlackTotal') / col('TotalPopulation'),4))\
    .withColumn('White_pct', round(col('WhiteTotal') / col('TotalPopulation'), 4))\
    .withColumn('Asian_pct', round(col('AsianTotal') / col('TotalPopulation'), 4))\
    .withColumn('Hispanic_pct', round(col('HispanicTotal') / col('TotalPopulation'),4))\
    .withColumn('AmericanIndianAlaskaNative_pct', round(col('AmericanIndianAlaskaNativeTotal') / col('TotalPopulation'),4))

In [23]:
demo2.show(2)

+-----------+---------+---------+--------------+----------------+---------------+--------------+------------+------------------+-------------------------------+----------+----------+-------------+----------+--------------+----------------+-----------------+-----------------+---------+---------+---------+------------+------------------------------+
|      State|StateCode|MedianAge|MalePopulation|FemalePopulation|TotalPopulation|NumberVeterans|Foreign-born|  AvgHouseholdSize|AmericanIndianAlaskaNativeTotal|AsianTotal|BlackTotal|HispanicTotal|WhiteTotal|MalePopPercent|FemalePopPercent|VeteranPopPercent|ForeignPopPercent|Black_pct|White_pct|Asian_pct|Hispanic_pct|AmericanIndianAlaskaNative_pct|
+-----------+---------+---------+--------------+----------------+---------------+--------------+------------+------------------+-------------------------------+----------+----------+-------------+----------+--------------+----------------+-----------------+-----------------+---------+---------+-----

This is a query to see if all states have one row and total population for each state

In [24]:
demo2.createOrReplaceTempView('demo')

In [25]:
# See how many rows
spark.sql(
"""
SELECT COUNT(*) as COUNT
FROM demo
"""
).show()

# See state code for first 10
spark.sql(
"""
SELECT StateCode, COUNT(*) as COUNT, SUM(TotalPopulation) as TotalPopulation
FROM demo
GROUP BY StateCode
ORDER BY COUNT DESC, TotalPopulation DESC
LIMIT 10
"""
).show(10)

# See which state codes have a count greater than 1
# Should return none. If not, then there is an error and should check again
spark.sql(
"""
SELECT StateCode, COUNT(*) as COUNT, SUM(TotalPopulation) as TotalPopulation
FROM demo
GROUP BY StateCode
HAVING COUNT > 1
ORDER BY COUNT DESC, TotalPopulation DESC
"""
).show()

+-----+
|COUNT|
+-----+
|   49|
+-----+

+---------+-----+---------------+
|StateCode|COUNT|TotalPopulation|
+---------+-----+---------------+
|       CA|    1|       24822460|
|       TX|    1|       14299983|
|       NY|    1|        9815626|
|       FL|    1|        6796738|
|       IL|    1|        4562312|
|       AZ|    1|        4499542|
|       NC|    1|        3060199|
|       CO|    1|        2935669|
|       WA|    1|        2500107|
|       OH|    1|        2433689|
+---------+-----+---------------+

+---------+-----+---------------+
|StateCode|COUNT|TotalPopulation|
+---------+-----+---------------+
+---------+-----+---------------+



#### 3) Airport Data Cleaning
- There are many nulls in the continent column. All are from "NA" values being processed as nulls. This is for the continent North America
- There are also nulls in the country column. These all have "NA" values being processed as nulls. This is for the country Namibia
- However, it appears that when using Spark these errors do not occur. Following is to double check that these are automatically read in correctly

In [26]:
# Import airport data into Spark dataframe
airports_spark = spark.read.csv('./airport-codes_csv.csv', header = True, inferSchema = True)

In [27]:
airports_spark.show(2)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
only showing top 2 rows



In [28]:
# Filter dataframe for only US airports
# Get the right two characters from iso_region column to get the state values
# Fix one erroneous value that has Asia as the continent value for a US airport
# Remove rows with us_state values of '-A', since EDA showed there are values with this value and is not a correct value
airports_spark = airports_spark\
    .filter(col('iso_country') =='US')\
    .filter(airports_spark.iata_code.isNotNull())\
    .withColumn('us_state', substring(airports_spark.iso_region,-2, 2))\
    .withColumn('continent', when((col('iso_country')=='US') & (col('continent')!='NA'), 'NA').otherwise(col('continent')))\
    .filter(col('us_state') != '-A')\
    .select('iata_code', 'name', 'type', 'elevation_ft', 'us_state')
    


In [29]:
airports_spark.show(2)

+---------+--------------------+-------------+------------+--------+
|iata_code|                name|         type|elevation_ft|us_state|
+---------+--------------------+-------------+------------+--------+
|      OCA|Ocean Reef Club A...|small_airport|           8|      FL|
|      PQS|Pilot Station Air...|small_airport|         305|      AK|
+---------+--------------------+-------------+------------+--------+
only showing top 2 rows



In [30]:
airports_spark.createOrReplaceTempView('airports')

In [31]:
# How many airports in this dataset?
spark.sql(
"""
SELECT count(iata_code) AS record_count
FROM airports
"""
).show()

# IS iata_code unique?
spark.sql(
"""
SELECT iata_code, count(iata_code) AS count
FROM airports
GROUP BY iata_code
HAVING count > 1
"""
).show()


# How many states are in this dataset?
spark.sql(
"""
SELECT count(distinct(us_state)) AS count
FROM airports
"""
).show()

# How many airports per state
spark.sql(
"""
SELECT us_state, count(iata_code) AS count
FROM airports
GROUP BY us_state
ORDER BY count DESC
"""
).show(52)

# State_code "-A" sticks out. Look at records with this state code 
# Query below should return nothing since filtered data on this parameter

spark.sql(
"""
SELECT *
FROM airports
WHERE us_state = '-A'
"""
).show()

+------------+
|record_count|
+------------+
|        2019|
+------------+

+---------+-----+
|iata_code|count|
+---------+-----+
|      CLG|    2|
|      AUS|    2|
|      AHT|    2|
|      ESP|    2|
|      PHL|    2|
+---------+-----+

+-----+
|count|
+-----+
|   51|
+-----+

+--------+-----+
|us_state|count|
+--------+-----+
|      AK|  334|
|      CA|  157|
|      TX|  122|
|      FL|   78|
|      WA|   55|
|      NY|   53|
|      MI|   48|
|      AZ|   46|
|      PA|   44|
|      OK|   43|
|      WI|   42|
|      IL|   41|
|      IA|   41|
|      OH|   38|
|      CO|   38|
|      IN|   37|
|      GA|   37|
|      OR|   36|
|      KS|   36|
|      MN|   35|
|      NC|   34|
|      VA|   32|
|      NE|   31|
|      SC|   31|
|      NV|   30|
|      AL|   30|
|      NM|   30|
|      WY|   29|
|      AR|   29|
|      MO|   29|
|      UT|   28|
|      MS|   27|
|      MT|   27|
|      TN|   26|
|      LA|   22|
|      MA|   22|
|      HI|   22|
|      ME|   19|
|      NJ|   18|
|     

#### 4a) Temperature Data (By State)
- Filter by Country = United States since we are interested in getting state temperatures of US states. This data <br>will be joined with the immigration data to provide information about the temperature of states that people are traveling to.
- Create a state abbreveation columns so that data can be joined using a standardized code.
- AverageTemperature column has the most recent temperature values with a date of 9/1/2013. 
- This data set is until 9/1/2013. However, immigration data is only for April 2016. Therefore, cannot join on individual dates <br>or even by year. Therefore, will be joining based on the average temperature for the state since the year 2000. Since <br>climate changes occur, having the most recent temperatures would be more useful.

In [32]:
# Import temperature data into Spark dataframe
state_temp = spark.read.csv('./GlobalLandTemperaturesByState.csv', header = True, inferSchema = True)

In [33]:
state_temp.show(2)

+-------------------+------------------+-----------------------------+-----+-------+
|                 dt|AverageTemperature|AverageTemperatureUncertainty|State|Country|
+-------------------+------------------+-----------------------------+-----+-------+
|1855-05-01 00:00:00|            25.544|                        1.171| Acre| Brazil|
|1855-06-01 00:00:00|            24.228|                        1.103| Acre| Brazil|
+-------------------+------------------+-----------------------------+-----+-------+
only showing top 2 rows



In [34]:
state_temp.printSchema()

root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)



In [35]:
# Create year, month, and country code columns
# filter by month = 4 since our fact table only has April 2016 data
# Want to see average temps for this month since 2000
state_temp_clean = state_temp\
    .filter(col('Country') == 'United States')\
    .withColumn('year', year(col('dt')))\
    .withColumn('month', month(col('dt')))\
    .withColumn('state_code', get_state_code(state_temp['State']))\
    .filter(col('year') > 1999)\
    .filter(col('month') == 4)

In [36]:
state_temp_clean.show(2)

+-------------------+------------------+-----------------------------+-------+-------------+----+-----+----------+
|                 dt|AverageTemperature|AverageTemperatureUncertainty|  State|      Country|year|month|state_code|
+-------------------+------------------+-----------------------------+-------+-------------+----+-----+----------+
|2000-04-01 00:00:00|            16.032|                         0.11|Alabama|United States|2000|    4|        AL|
|2001-04-01 00:00:00|            18.517|                        0.129|Alabama|United States|2001|    4|        AL|
+-------------------+------------------+-----------------------------+-------+-------------+----+-----+----------+
only showing top 2 rows



In [37]:
state_temp_agg = state_temp_clean\
    .groupBy('State','state_code')\
    .agg(round(f.avg('AverageTemperature'),2).alias('AvgTemp'))

In [38]:
state_temp_agg.show(2)

+------------+----------+-------+
|       State|state_code|AvgTemp|
+------------+----------+-------+
| Mississippi|        MS|  18.38|
|South Dakota|        SD|   7.74|
+------------+----------+-------+
only showing top 2 rows



In [39]:
state_temp_agg.createOrReplaceTempView('state_temp')

In [40]:
# See how many states there are. Should be 51 (50 states and Washington + DC)
spark.sql(
"""
SELECT COUNT(DISTINCT(State)) AS TotalStates
FROM state_temp
"""
).show()

# See if states have more than one row. Should return none/be blank
spark.sql(
"""
SELECT State, count(AvgTemp) AS COUNT
FROM state_temp
GROUP BY State
HAVING COUNT > 1
"""
).show()

+-----------+
|TotalStates|
+-----------+
|         51|
+-----------+

+-----+-----+
|State|COUNT|
+-----+-----+
+-----+-----+



#### 4b) Temperature Data (By Country)
- Notes are similar to part 4a for state temperatures. However, need to give countries a country code instead for joining.
- This data will be used to provide temperature information about countries outside of the US where travelers are coming from. 

In [41]:
country_temp = spark.read.csv('./GlobalLandTemperaturesByCountry.csv', header = True, inferSchema = True)

In [42]:
# Create year, month, and country code columns
# Filter out non-US columns since the country temperature data will be joined using i94res from the immigration data. 
# i94res represents the country where the person is coming from
country_temp_clean = country_temp\
    .filter(col('AverageTemperature').isNotNull())\
    .withColumn('year', year(col('dt')))\
    .withColumn('month', month(col('dt')))\
    .withColumn('country_code', get_country_code(upper(col('Country'))))\
    .filter(col('year') > 1999)\
    .filter(col('month') == 4)\
    .filter(col('country_code') != '99999')

In [43]:
country_temp_clean.show(2)

+-------------------+------------------+-----------------------------+-----------+----+-----+------------+
|                 dt|AverageTemperature|AverageTemperatureUncertainty|    Country|year|month|country_code|
+-------------------+------------------+-----------------------------+-----------+----+-----+------------+
|2000-04-01 00:00:00|             18.44|                        0.522|Afghanistan|2000|    4|          AF|
|2001-04-01 00:00:00|             17.23|           0.5720000000000001|Afghanistan|2001|    4|          AF|
+-------------------+------------------+-----------------------------+-----------+----+-----+------------+
only showing top 2 rows



In [44]:
country_temp_agg = country_temp_clean\
    .groupBy('Country','country_code')\
    .agg(round(f.avg('AverageTemperature'), 2).alias('AvgTemp'))

In [45]:
country_temp_agg.show(2)

+---------+------------+-------+
|  Country|country_code|AvgTemp|
+---------+------------+-------+
|   Sweden|          SE|   2.72|
|Indonesia|          ID|  26.48|
+---------+------------+-------+
only showing top 2 rows



In [46]:
country_temp_agg.createOrReplaceTempView('country_temp')

In [47]:
# Verify one country per row. Should not have groups with counts greater than 1 and return none
spark.sql(
"""
SELECT Country, count(AvgTemp) AS COUNT
FROM country_temp
GROUP BY Country
HAVING COUNT > 1
"""
).show()


# Get the number of countries contained in the dataset. Filtered for countries without a country code
spark.sql(
"""
SELECT count(distinct(Country)) AS COUNT
FROM country_temp
"""
).show()

+-------+-----+
|Country|COUNT|
+-------+-----+
+-------+-----+

+-----+
|COUNT|
+-----+
|  220|
+-----+



### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The data model uses a star schema with fact and dimension tables. A star schema is a simple schema design that makes performing analytics queries very easy. An analytics team will benefit from using a star schema by allowing for simpler queries with easier to understand join logic from the business perspective. It will also boost query performance.

The final star schema data model will have the following fact and dimension tables (also available in data dictioanry with more detail):

__FACT TABLE__: 

- id, Month, Year,Arrival_Date, Departure_Date, Trip_Duration, Visa_Type, Gender, Age, <br>US_State_Name, State_Temp, State_Population, State_Male_Population, State_Female_Population, <br>Origin_Country, Origin_Country_Temp, Airport_Name, Airport_State, Airline, Mode_Transportation

<br>

__DIMENSION TABLES__:

__*Immigration Table*__:
- id, month, year, arrival_date, departure_date, trip_duration, origin_country_code, origin_country,<br> transportation_mode, us_state, us_state_name, port, gender, age, airline, visa_type


__*Airport Table*__:
- iata_code, name, type, elevation_ft, us_state

__*Demographics Table*__:
- State, StateCode, MedianAge, MalePopulation, FemalePopulation, TotalPopulation, NumberVeterans, <br>Foreign-born, AvgHouseholdSize, AmericanIndianAlaskaNativeTotal, AsianTotal, BlackTotal,<br> HispanicTotal, WhiteTotal, MalePopPercent, FemalePopPercent, VeteranPopPercent, TotalPopulation,<br> ForeignPopPercent, Black_pct, White_pct, Asian_pct, Hispanic_pct, AmericanIndianAlaskaNative_pct

__*State Temperature Table*__:
- State, state_code, AvgTemp 
__*Country Temperature Table*__:
- Country, country_code, AvgTemp

<br>

#### 3.2 Mapping Out Data Pipelines
The following are the steps needed to pipeline the data into the data model
1. Create dimension tables using the cleaned data
2. Create the fact table from the cleaned immigration data
3. Write fact table to a parquet file

<br>

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [48]:
# Create temporary views to allow the use of Spark SQL

immigration_clean.createOrReplaceTempView('immigration')
demo2.createOrReplaceTempView('demo')
airports_spark.createOrReplaceTempView('airports')
state_temp_agg.createOrReplaceTempView('state_temp')
country_temp_agg.createOrReplaceTempView('country_temp')

In [49]:
immigration_clean.printSchema()

root
 |-- id: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- trip_duration: integer (nullable = true)
 |-- origin_country_code: string (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- transportation_mode: integer (nullable = true)
 |-- us_state: string (nullable = true)
 |-- us_state_name: string (nullable = true)
 |-- port: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- visa_type: integer (nullable = true)



In [50]:
airports_spark.printSchema()

root
 |-- iata_code: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- us_state: string (nullable = true)



In [51]:
demo2.printSchema()

root
 |-- State: string (nullable = true)
 |-- StateCode: string (nullable = true)
 |-- MedianAge: double (nullable = true)
 |-- MalePopulation: long (nullable = true)
 |-- FemalePopulation: long (nullable = true)
 |-- TotalPopulation: long (nullable = true)
 |-- NumberVeterans: long (nullable = true)
 |-- Foreign-born: long (nullable = true)
 |-- AvgHouseholdSize: double (nullable = true)
 |-- AmericanIndianAlaskaNativeTotal: long (nullable = true)
 |-- AsianTotal: long (nullable = true)
 |-- BlackTotal: long (nullable = true)
 |-- HispanicTotal: long (nullable = true)
 |-- WhiteTotal: long (nullable = true)
 |-- MalePopPercent: double (nullable = true)
 |-- FemalePopPercent: double (nullable = true)
 |-- VeteranPopPercent: double (nullable = true)
 |-- ForeignPopPercent: double (nullable = true)
 |-- Black_pct: double (nullable = true)
 |-- White_pct: double (nullable = true)
 |-- Asian_pct: double (nullable = true)
 |-- Hispanic_pct: double (nullable = true)
 |-- AmericanIndianAlask

In [52]:
state_temp_agg.printSchema()

root
 |-- State: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- AvgTemp: double (nullable = true)



In [53]:
country_temp_agg.printSchema()

root
 |-- Country: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- AvgTemp: double (nullable = true)



In [54]:
# Selecting fields and joining datasets

immigration_final = spark.sql(
"""
SELECT 
       i.id AS id,
       i.year AS Year,
       i.month AS Month,
       i.arrival_date AS Arrival_Date,
       i.departure_date AS Departure_Date,
       i.trip_duration AS Trip_Duration,
       i.visa_type AS Visa_Type, 
       i.gender AS Gender,
       i.age AS Age,
       i.us_state_name AS US_State_Name,
       s.AvgTemp AS State_Temp,
       d.TotalPopulation AS State_Population,
       d.MalePopPercent AS State_Male_Population,
       d.FemalePopPercent AS State_Female_Population,
       i.origin_country as Origin_Country,
       c.AvgTemp AS Origin_Country_Temp,
       a.name AS Airport_Name,
       a.us_state AS Airport_State,
       i.airline AS Airline,
       i.transportation_mode AS Mode_Transportation
FROM immigration i 
    JOIN demo d on i.us_state = d.StateCode
    JOIN airports a on i.port = a.iata_code
    JOIN state_temp s on i.us_state = s.state_code
    JOIN country_temp c on i.origin_country_code = c.country_code
"""
)

In [55]:
immigration_final.show(1)

+-------+----+-----+------------+--------------+-------------+---------+------+---+-------------+----------+----------------+---------------------+-----------------------+--------------+-------------------+--------------------+-------------+-------+-------------------+
|     id|Year|Month|Arrival_Date|Departure_Date|Trip_Duration|Visa_Type|Gender|Age|US_State_Name|State_Temp|State_Population|State_Male_Population|State_Female_Population|Origin_Country|Origin_Country_Temp|        Airport_Name|Airport_State|Airline|Mode_Transportation|
+-------+----+-----+------------+--------------+-------------+---------+------+---+-------------+----------+----------------+---------------------+-----------------------+--------------+-------------------+--------------------+-------------+-------+-------------------+
|5748525|2016|    4|  2016-04-30|    2016-05-07|            7|        2|     M| 27|      Florida|     21.49|         6796738|               0.4762|                 0.5131|   NEW ZEALAND|    

In [56]:
immigration_final.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Arrival_Date: date (nullable = true)
 |-- Departure_Date: date (nullable = true)
 |-- Trip_Duration: integer (nullable = true)
 |-- Visa_Type: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- US_State_Name: string (nullable = true)
 |-- State_Temp: double (nullable = true)
 |-- State_Population: long (nullable = true)
 |-- State_Male_Population: double (nullable = true)
 |-- State_Female_Population: double (nullable = true)
 |-- Origin_Country: string (nullable = true)
 |-- Origin_Country_Temp: double (nullable = true)
 |-- Airport_Name: string (nullable = true)
 |-- Airport_State: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Mode_Transportation: integer (nullable = true)



Write dataframe to parquet file:

In [57]:
#write to parquet
immigration_final.write.parquet("immigration_fact_table", mode = 'overwrite')

In [58]:
# read back data to double check
spark.read.parquet("immigration_fact_table").show(3)

+-------+----+-----+------------+--------------+-------------+---------+------+---+-------------+----------+----------------+---------------------+-----------------------+--------------+-------------------+--------------------+-------------+-------+-------------------+
|     id|Year|Month|Arrival_Date|Departure_Date|Trip_Duration|Visa_Type|Gender|Age|US_State_Name|State_Temp|State_Population|State_Male_Population|State_Female_Population|Origin_Country|Origin_Country_Temp|        Airport_Name|Airport_State|Airline|Mode_Transportation|
+-------+----+-----+------------+--------------+-------------+---------+------+---+-------------+----------+----------------+---------------------+-----------------------+--------------+-------------------+--------------------+-------------+-------+-------------------+
|5748525|2016|    4|  2016-04-30|    2016-05-07|            7|        2|     M| 27|      Florida|     21.49|         6796738|               0.4762|                 0.5131|   NEW ZEALAND|    

### Possible Analytic Queries

The following are possible analytic queries that can be performed using the fact table above:

In [59]:
immigration_final.createOrReplaceTempView('final')

In [60]:
# For the given timeframe of the dataset, how many people immigrated on each type of visa?

print('In April 2016, people arrived in the United States with the following types of visas:')

spark.sql(
"""
SELECT Visa_Type, count(Year) as Count
FROM final
GROUP BY Visa_Type
ORDER BY Visa_Type
"""
).show(4)
print('Visa types are:\n1 = Business\n2 = Pleasure\n3 = Student\nAppears most people traveling for pleasure')

# The top five airports that travlers came through were the following:

print('Top five airports people traveled through:')

spark.sql(
"""
SELECT Airport_Name AS Airport, count(Year) as Count
FROM final
GROUP BY Airport_Name
ORDER BY Count DESC
LIMIT 5
"""
).show(5)



In April 2016, people arrived in the United States with the following types of visas:
+---------+-------+
|Visa_Type|  Count|
+---------+-------+
|        1| 268572|
|        2|1061322|
|        3|  18726|
+---------+-------+

Visa types are:
1 = Business
2 = Pleasure
3 = Student
Appears most people traveling for pleasure
Top five airports people traveled through:
+--------------------+------+
|             Airport| Count|
+--------------------+------+
|Miami Internation...|325002|
|San Fernando Airport|147535|
|Orlando Executive...|142905|
|   Lakefront Airport|132360|
|William P Hobby A...| 94591|
+--------------------+------+



In [61]:
# The top countries that people came from

print('The top countries that people immigrated from:')

spark.sql(
"""
SELECT Origin_Country, count(Year) as Count
FROM final
GROUP BY Origin_Country
ORDER BY Count DESC
LIMIT 5
"""
).show()


The top countries that people immigrated from:
+--------------------+------+
|      Origin_Country| Count|
+--------------------+------+
|      UNITED KINGDOM|181801|
|MEXICO Air Sea, a...| 95298|
|              BRAZIL| 86469|
|             GERMANY| 85277|
|              FRANCE| 75344|
+--------------------+------+



In [62]:
# UNITED KINGDOM was the country most visitors came. They went to the following states most often:

print('The country where most visitors came from was the United Kingdom. They traveled to the following states most often:')

spark.sql(
"""
SELECT US_State_Name, count(Year) as Count
FROM final
WHERE Origin_Country = 'UNITED KINGDOM'
GROUP BY US_State_Name
ORDER BY Count DESC
LIMIT 5
"""
).show()

The country where most visitors came from was the United Kingdom. They traveled to the following states most often:
+-------------+-----+
|US_State_Name|Count|
+-------------+-----+
|      Florida|85143|
|   California|18322|
|     New York|18120|
|        Texas|12563|
|Massachusetts| 7441|
+-------------+-----+



In [68]:
# What is the difference in temperature between the UK and the US states traveled to
# People could be traveling due to warm weather vacactions
spark.sql(
"""
SELECT US_State_Name, count(Year) as Count,
    round(avg(Origin_Country_Temp),2) AS UK_Temp, round(avg(State_Temp),2) AS State_Temp,
    round((avg(Origin_Country_Temp) - avg(State_Temp)), 2) AS Temp_Difference
FROM final
WHERE Origin_Country = 'UNITED KINGDOM'
GROUP BY US_State_Name
ORDER BY Temp_Difference
LIMIT 5
"""
).show()

+-------------+-----+-------+----------+---------------+
|US_State_Name|Count|UK_Temp|State_Temp|Temp_Difference|
+-------------+-----+-------+----------+---------------+
|       Hawaii|  724|    8.2|     21.59|         -13.39|
|      Florida|85143|    8.2|     21.49|         -13.29|
|    Louisiana| 1659|    8.2|     19.86|         -11.66|
|        Texas|12563|    8.2|     19.15|         -10.95|
|  Mississippi|  121|    8.2|     18.38|         -10.18|
+-------------+-----+-------+----------+---------------+



In [69]:
# Based on temperature differences, people may travel to Florida from the UK for vacation
# Look at types of visas to see what type of travel is being made to Florida
spark.sql(
"""
SELECT Visa_Type, count(Year) as Count
FROM final
WHERE Origin_Country = 'UNITED KINGDOM' AND US_State_Name = 'Florida'
GROUP BY Visa_Type
ORDER BY Count DESC
"""
).show()

print('Visa types are:\n1 = Business\n2 = Pleasure\n3 = Student\n\nAppears most people travel to Florida for pleasure')

+---------+-----+
|Visa_Type|Count|
+---------+-----+
|        2|80786|
|        1| 4305|
|        3|   52|
+---------+-----+

Visa types are:
1 = Business
2 = Pleasure
3 = Student

Appears most people travel to Florida for pleasure


#### 4.2 Data Quality Checks
Two data quality tests:

__Test 1__

Check if all the primary keys used in the JOINs are unique. Since a primary key is a unique identifier, there should only be one of each in the column. This tests counts the number of times a category or value occurs in the primary key column, and will return values that do not pass the test of a unqiue value. If the tests return blank values then it passes.

In [65]:
# Re-run code to create views for Spark SQL queries
immigration_clean.createOrReplaceTempView('immigration')
demo2.createOrReplaceTempView('demo')
airports_spark.createOrReplaceTempView('airports')
state_temp_clean.createOrReplaceTempView('state_temp')
country_temp_clean.createOrReplaceTempView('country_temp')

In [66]:
# Test unique keys. 
# In order to pass, all queries should return none 

# StateCode from demographics test
spark.sql(
"""
SELECT StateCode, count(distinct(StateCode)) AS Count
FROM demo
GROUP BY StateCode
HAVING Count > 1
ORDER BY StateCode DESC
"""
).show()


# iata_code from airports test
spark.sql(
"""
SELECT iata_code, count(distinct(iata_code)) AS Count
FROM airports
GROUP BY iata_code
HAVING Count > 1
ORDER BY iata_code DESC
"""
).show()

# state_code from state temperatures test
spark.sql(
"""
SELECT state_code, count(distinct(state_code)) AS Count
FROM state_temp
GROUP BY state_code
HAVING Count > 1
ORDER BY state_code DESC
"""
).show()

# country_code from country temperatures test
spark.sql(
"""
SELECT country_code, count(distinct(country_code)) AS Count
FROM country_temp
GROUP BY country_code
HAVING Count > 1
ORDER BY country_code DESC
"""
).show()

+---------+-----+
|StateCode|Count|
+---------+-----+
+---------+-----+

+---------+-----+
|iata_code|Count|
+---------+-----+
+---------+-----+

+----------+-----+
|state_code|Count|
+----------+-----+
+----------+-----+

+------------+-----+
|country_code|Count|
+------------+-----+
+------------+-----+



__Test 2__

This test is to check if datatypes are still correct. Will check the date fields since these are the ones that changed from numbers to date. Want to ensure these data type conversions made it through the data wrangling process

In [67]:
immigration_final.dtypes[2:4]

[('Month', 'int'), ('Arrival_Date', 'date')]

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.


See markdown file 'Data Dictionary.md' for 

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

__Responses to Prompts__:

I decided to use Spark since it is a framework for processing large amounts of data. This slice of the data was for one month's worth of data. It contained approximately 3 million rows and performing the work locally worked sufficiently. However, if we needed to process a year's worth of data or more, we would need more computing power. Since we already have the data pipeline coded using Spark's framework and syntax, we could quickly implement the pipeline using on premise or cloud based distributed computing solutions. 

I propose to update the data on a monthly basis. This would allow for monthly analysis of immigration trends and to make decisions in a timely fashion. This data is not like a sales transaction database that needs to be processed much more frequently, and therefore the frequency of updates is sufficient. Under the proposal of monthly updates, analysts can still perform monthly reporting tasks while keeping costs low by processing the data 12 times a year.

I would handle the problem differently in the following when ways when presented with new challenges outline above:
- Use an AWS EMR  to process larger amounts of data. This notebook only performed data pipeline <br> activities on one month of data so I was able to perform the tasks locally. If there is much more <br>data to be processed, utilizing distributed clusters on a cloud platform <br>would make much more sense and execute much faster. This would be much cheaper than investing <br>in on premise solutions and enable us to begin work much quicker.
- I would use an automated scheduler/workflow manager such as Airflow. Airflow would allow us to schedule <br>ETL processes to execute in an automated fashion and provides great monitoring and data quality capabilities.
- I would use a cloud datawarehouse product like Amazon Redshift. Since the data warehouse is in the cloud, <br>it is scalable solution if a lot of people need access or need to perform their own ad hoc analyses. It would also be <br>simple since each person can access it with their own credentials and perform queries. Redshift also<br> is capable of making connections to other tools like Jupyter, data visualization tools (Power BI, Tableau, Looker,etc.) 